Gerekli kütüphaneleri ekleyelim.

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import missingno           

from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plot 
import os
from subprocess import check_output
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from warnings import filterwarnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix as cm
from sklearn import ensemble
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
filterwarnings('ignore')

sns.set(rc={'figure.figsize':(10,8)})

Verisetimizi ekleyelim ve df değişkenine atayalım.

In [ ]:
df = pd.read_csv('../input/tabletpc-priceclassification/tablet.csv')

In [ ]:
df

İlk 5 gözlemi görüntüleyelim

In [ ]:
df.head()

In [ ]:
df.shape

20 özniteliğe sahip 2000 adet tablet bulunuyor.
Şimdi kaç gözlem var, ne tür öznitelikler bulunuyor, eksik gözlem var mı gibi sorulara yanıt arayalım

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

2000 tane gözlemden 5 tanesinin OnKameraMP, 12 tanesinin RAM öznitelik bilgisi yok.

Veri çerçevemizin istatistiksel değerlerini görüntüleyelim.

In [ ]:
df.describe()

OnKameraMP özniteliğinin standart sapmasının ortalamadan büyük olması dikkat çekiyor. Dağınık bir öznitelik diyebiliriz.

Hedef değişken hangisidir?
Hedef değişkenimiz burada tabletin ana ve diğer özelliklerine bağlı olarak değişen fiyat aralığıdır. Çünkü istediğimiz şey bu veri seti ile tablet cihazlarının fiyat analizini yapmaktır.

In [ ]:
df["FiyatAraligi"].unique()

4 adet "fiyat aralığı" tipimiz var.

In [ ]:
df.FiyatAraligi.value_counts()

Tüm fiyat aralıkları eşit olarak dağılmış.

In [ ]:
sns.countplot(x = "FiyatAraligi", data = df);

In [ ]:
df.hist()

In [ ]:
sns.scatterplot(x = "CozunurlukGenislik", y = "CozunurlukYükseklik", hue = "FiyatAraligi", data = df); #Çözünürlük genişlik ve yükseklik arasında da genel olarak doğrusal bir ilişki var.

In [ ]:
sns.scatterplot(x = "ArkaKameraMP", y = "OnKameraMP", hue = "FiyatAraligi", data = df); #Örneğin ön kamera ve arka kamera arasındaki fiyat aralığı dağılımına bakalım. Değerler üçgen şeklinde artarak seyrediyor. Arka kamera çözünürlüğü arttıkça ön kamera çözünürlüğü de artıyor. Doğru orantısız değerler de var.

İkisinde de doğru orantı görüyoruz.
Kameralar üçgene benzer şekilde artıyor.

In [ ]:
sns.scatterplot(x = "BataryaGucu", y = "BataryaOmru", hue = "FiyatAraligi", data = df); 

Batarya gücü ve batarya ömrü ilgisiz değerler.

In [ ]:
sns.distplot(df["RAM"], bins=32, color="red");

Ram değerleri arasında benzer bir dağılım söz konusu. Fiyat aralığı gibi. Şimdi bunu ayrı bir grafikte görelim.

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "RAM", data = df);

Fiyat ve RAM arasında doğru orantı olduğunu söylemek mümkün.

In [ ]:
sns.relplot(x='OnKameraMP', y='ArkaKameraMP', hue='FiyatAraligi', size='FiyatAraligi', col='FiyatAraligi', data=df)

Şimdi eksik veriler arasında bir ilişki var mı, eksik verileri nasıl doldurabiliriz gibi sorulara cevap arayacağız.

In [ ]:
missingno.matrix(df,figsize=(20, 10)); 

Her bir beyaz çizgi eksik veriyi temsil ediyor.

In [ ]:
missingno.heatmap(df, figsize= (20,10));

Eksik verilerin biribiryle ilişkisi olmadığını görebiliriz. Ayrıca eksik veriler gözlemlerin yüzde olarak çok küçük bir kısmını oluşturuyorlar. Yüzde olarak 1'in altındalar. Yani bir çıkarımda bulunmak için ciddiye alınacak bir yüzdede değiller.

Şimdi eksik verileri doldurma işlemine geçelim.

In [ ]:
df.groupby("FiyatAraligi").mean()

Groupby komutu ile bir ilişki bulmaya çalışacağız.

In [ ]:
df.groupby("FiyatAraligi")[["OnKameraMP"]].mean() 

Ön kamera çözünürlüğü eksik olan fiyat aralıklarına bakalım.

In [ ]:
sns.countplot(df[df["OnKameraMP"].isnull()]["FiyatAraligi"]);

Sadece çok ucuz fiyat aralığındaki ön kamera bilgileri eksik.

In [ ]:
cok_ucuz_OnKameraMP = df[(df["FiyatAraligi"] == "Çok Ucuz") & (df["OnKameraMP"].isnull())].index
cok_ucuz_OnKameraMP

In [ ]:
df.loc[cok_ucuz_OnKameraMP,"OnKameraMP"] = 4

Eksik değerleri 4 ile doldurmak mantıklı olabilir çünkü fiyat aralığındaki ortalama değer 4'e çok yakın.

In [ ]:
df.isna().sum()["OnKameraMP"]

İşlemimiz başarılı eksik veriyi 0 gösteriyor.

Şimdi RAM özniteliğindeki eksik veriye bir göz atalım.

In [ ]:
df.groupby("FiyatAraligi")[["RAM"]].mean() 

In [ ]:
sns.countplot(df[df["RAM"].isnull()]["FiyatAraligi"]);

Burada da pahalı fiyat aralığındaki RAM bilgisinde eksikler görmekteyiz.

In [ ]:
df[(df["FiyatAraligi"] == "Pahalı") & (df["RAM"].isnull())]

In [ ]:
pahali_RAM = df[(df["FiyatAraligi"] == "Pahalı") & (df["RAM"].isnull())].index
pahali_RAM

In [ ]:
df.loc[pahali_RAM,"RAM"] = 3450

Eksik verilerimizi ortalamayı göz önünde bulundurarak 3450 ile doldurduk.

In [ ]:
df.isna().sum()["RAM"]

İşlemimizin başarılı olduğunu görüyoruz.

Sonuç olarak eksik verilerin fiyat aralığına bağlı olduğunu gördük ve ortalama değerlerini kullanarak bu eksikleri kapattık. Eksik veri sayısı toplam veri sayısına oranla çok küçük bir sayı olduğu için olası hataları önemsemeyeceğiz.

Test train split için bazı verileri sayısallaştırmamız gerekecek. Bunun preprocessing kutüphanesinden yararlanacağız.

In [ ]:
le = preprocessing.LabelEncoder()


In [ ]:
df["Bluetooth"] = le.fit_transform(df["Bluetooth"])
df.head()

Burada 1=hayır 0=evet olduğuna dikkat edelim.

In [ ]:
df["CiftHat"] = le.fit_transform(df["CiftHat"])
df.head()

In [ ]:
df["4G"] = le.fit_transform(df["4G"])
df.head()

In [ ]:
df["Dokunmatik"] = le.fit_transform(df["Dokunmatik"])
df.head()

In [ ]:
df["3G"] = le.fit_transform(df["3G"])
df.head()

In [ ]:
df["WiFi"] = le.fit_transform(df["WiFi"])
df.head()

In [ ]:
df["Renk_Encoded"] = le.fit_transform(df["Renk"])
df.head()

Renk özniteliğinin çok fazla çeşidi olduğunu biliyoruz. (12) Bu yüzden bu değeri ayrı bir öznitelik olarak tanımladık ki kolay bir şekilde hangi rengin ne ifade ettiğini anlayabilelim.

In [ ]:
df["FiyatAraligi_Encoded"] =  le.fit_transform(df["FiyatAraligi"])
df.head()

In [ ]:
df.groupby(["FiyatAraligi"]).mean()

In [ ]:
df.groupby(["FiyatAraligi"]).std() 

Korelasyon matrisini çizdireceğiz

In [ ]:
df.corr()

Çoğu değerin negatif yönde bir ilişkisi var ve 1 'e yakın pozitif yönde ilişki çok az.

In [ ]:
df.corr()["ArkaKameraMP"]["OnKameraMP"]

En güçlü pozitif ilişki arka kamera ve ön kamera arasında.

In [ ]:
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

Çoğu öznitelik arasında ilişki olkmadığını rahatlıkla söyleyebiliriz.

Sayısal sütunları ve 1den çok benzersiz sütunları saklayarak bir çekirdek yoğunluğu grafiği çizdirelim.

In [ ]:
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) 
 
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] 
    columnNames = list(df)
    if len(columnNames) > 10: 
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plot.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plot.suptitle('Scatter and Density Plot')
    plot.show()


In [ ]:
plotScatterMatrix(df,20,10)

Grafiğin belirli yerlerinde azalıp arttığını gördüğümüz yoğunluk korelasyon matrisine bağlı olarak gerçekleşiyor.

Şimdi regresyon tahmini yapmak için modelimizi oluşturacağız.

In [ ]:
lineer_regresyon = LinearRegression() 

In [ ]:
X = df.drop(["FiyatAraligi_Encoded","FiyatAraligi","Renk"], axis = 1)
y = df["FiyatAraligi_Encoded"] 

Renk özniteliğini "dropladık" çünkü object değer içeriyordu ve float türüne dönüştüremiyorduk. Bu yüzden encoded versiyonunu kullanacağız.

In [ ]:
X

Bağımlı değişkeni etkileyen öznitelikler yani bağımsız değişkenler.

In [ ]:
y

Bağımlı değişken. Özelliklere göre farklılık gösteriyor.

In [ ]:
lineer_regresyon.fit(X, y)

In [ ]:
lineer_regresyon.predict([[1233,1,1,1,2,0,50,0.1,24,1,3,428,695,2000,2,0,1,1,0,]]) 

In [ ]:
df["model1_prediction"] = lineer_regresyon.predict(X)
df

Rastgele değerlerle oluşturulan tahmin modeli gerçek değerle yakınlık gösteriyor diyebiliriz.

Bağımlı ve bağımsız değişkenleri train ve test olarak ayıralım.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25, 
                                                    random_state = 42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

Şimdi eğittiğimiz değişkenleri kullanarak GaussianNB Modelinde işlemler yapacağız.

In [ ]:
NB = GaussianNB()
NB_model = NB.fit(X_train,y_train)

In [ ]:
NB_model

Modelin eğitimde kullanılmayan x testin ilk 20 gözleminin fiyat aralığını tahmin ettirmeye çalışalım.

In [ ]:
X_test[0:20]

In [ ]:
NB_model.predict(X_test)[0:20]

Şimdi gerçek veriye bakalım

In [ ]:
y_test[0:20]

Doğru olmayan tahminler göze çarpıyor.

In [ ]:
y_pred = NB_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_test

y_pred adlı bir değişken oluşturup X_test tahmin sonuçlarını bu değişkene atadık.

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
karmasiklik_matrisi

In [ ]:
(karmasiklik_matrisi[0][0] + karmasiklik_matrisi[1][1]) / (karmasiklik_matrisi[0][0] + karmasiklik_matrisi[1][1] +  karmasiklik_matrisi[1][0] + karmasiklik_matrisi[0][1])

Gerçek matematiksel işlem ve tahmin arasında 0,06 kadar bir hata payı olduğunu görüyoruz.

In [ ]:
cross_val_score(NB_model, X_test, y_test, cv = 20)

In [ ]:
cross_val_score(NB_model, X_test, y_test, cv = 20).mean()

In [ ]:
print(classification_report(y_test, y_pred))

Sınıflandırma raporu tablosunda 0.72 - 0.92 arasında değişkenlik gösteriyor.

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore

Decision tree modelimizi oluşturalım ve eğitelim.

In [ ]:
cart = DecisionTreeClassifier(random_state = 42, criterion='gini')
cart_model_gini = cart.fit(X_train, y_train)

In [ ]:
cart_model_gini

In [ ]:
y_pred_gini = cart_model_gini.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred_gini)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred_gini)
print(karmasiklik_matrisi)

Değerlerin dengesiz olduğunu görmek mümkün.

In [ ]:
cross_val_score(cart_model_gini, X, y, cv = 20)

20 tane değer için birbirine çok yakın olduğunu görüyoruz. Sayılar arasındaki fark birbirine yakın değerde diyebiliriz.

In [ ]:
cross_val_score(cart_model_gini, X, y, cv = 20).mean()

In [ ]:
print(classification_report(y_test, y_pred_gini))

Yakın hatta bazen aynı değerleri görmekteyiz.

Şimdi gini criterion değerini entropy olarak değiştirip skor değerlerini buna göre görüntüleyeceğiz.

In [ ]:
cart = DecisionTreeClassifier(random_state = 42, criterion='entropy')
cart_model = cart.fit(X_train, y_train)

In [ ]:
cart_model

In [ ]:
y_pred2 = cart_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred2)

0.816'dan 0,852'ye bir artış görüyoruz. 0,036 kadar bir artış söz konusu. Entropy ile daha başarılı bir sonuç elde ettik.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred2)
print(karmasiklik_matrisi)

Dengesiz ancak simetrik olarak yakın veriler görüyoruz.

In [ ]:
cross_val_score(cart_model, X, y, cv = 20)

In [ ]:
cross_val_score(cart_model, X, y, cv = 20).mean()

Yine az da olsa bir artış söz konusu.

In [ ]:
print(classification_report(y_test, y_pred2))

In [ ]:
graph = Source(tree.export_graphviz(cart, out_file = None, feature_names = X.columns, filled = True))
display(SVG(graph.pipe(format = 'svg')))

Karar ağacımızın RAM özniteliği doğrultusunda karar verdiğini görmek mümkün.

Özniteliklerin önem derecesini sıralayalım.

In [ ]:
ranking = cart.feature_importances_
features = np.argsort(ranking)[::-1][:20]
columns = X.columns

plot.figure(figsize = (16, 9))
plot.title("Karar Ağacına Göre Özniteliklerin Önem Derecesi", y = 1.03, size = 18)
plot.bar(range(len(features)), ranking[features], color="purple", align="center")
plot.xticks(range(len(features)), columns[features], rotation=80)
plot.show()

Bir fiyatlandırma yapacak olsak RAM değerine en fazla bakılması gerektiği tabloda bize anlatılıyor.

KNN modeliyle skor değerlerini normal gözlemlerle inceleyelim.

In [ ]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)

In [ ]:
knn_model

In [ ]:
y_pred3 = knn_model.predict(X)

In [ ]:
accuracy_score(y, y_pred3)

(Tüm özellikler için doğruluk skoru.)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y, y_pred3)
print(karmasiklik_matrisi)

Köşegen

In [ ]:
cross_val_score(knn_model, X_test, y_test, cv = 20)

In [ ]:
cross_val_score(knn_model, X_test, y_test, cv = 20).mean()

In [ ]:
print(classification_report(y, y_pred3))

En başarılı modelin bu olduğunu rahatlıkla söyleyebiliriz.

In [ ]:
knn_params = {"n_neighbors": np.arange(2,15)}
knn_params

En yakın komşu sayısını 2 ile 15 arasında deneyecek.

In [ ]:
knn_komsu = KNeighborsClassifier()
knn_cv = GridSearchCV(knn_komsu, knn_params, cv = 3)
knn_cv.fit(X_train, y_train)

In [ ]:
print("En iyi skor: " + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

In [ ]:
knn = KNeighborsClassifier(9)
knn_tuned = knn.fit(X_train, y_train)

In [ ]:
y_pred = knn_tuned.predict(X_test) 

In [ ]:
accuracy_score(y_test, y_pred)

Test ve ayarlanmış değerler için  doğruluk skoru. En iyi skor ile aynı olmasına dikkat edelim.

Doğruluk skorlarını görselleştirelim.

In [ ]:
skor_listesi = []

for each in range(2,15):
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(X_train,y_train)
    skor_listesi.append(knn2.score(X_test, y_test))
    
plot.plot(range(2,15),skor_listesi)
plot.xlabel("k değerleri")
plot.ylabel("doğruluk skoru")
plot.show()

Model için en iyi komşu olarak ayarlanmış gözlem ile skorunu tekrar hesaplayalım.

In [ ]:
cross_val_score(knn_tuned, X_test, y_test, cv = 20)


In [ ]:
cross_val_score(knn_tuned, X_test, y_test, cv = 20).mean()

Önceki normal cross val skoru normal gözlemler için 0.8870898 iken komşu sayılarıyla birlikte tekrar gözlemlediğimizde 0.9047537 çıktı.

KNN modelinin bu veriseti için daha doğru bir analiz aracı olduğunu rahatlıkla söyleyebiliriz. Doğruluk skorlarında yaptığımız karşılaştırmalarda da bunu net bir şekilde görmekteyiz.